In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import EmbeddingsModel
from trainer import EmbeddingTrainer, all_subjects, AccuracyTester, all_subjects_intense
from pytorch_metric_learning import distances


import csv
import pandas as pd
from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

setup parameters

In [7]:
use_model = True

saving_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\PainClassificationTest\Results-10-fold-crossval\\xite_tonic_electric2.csv"

In [3]:
#init file
with open(saving_path, mode='w') as result_file:
    results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    results_writer.writerow(["subject", "accuracy"])

get all not processed subjects

In [8]:
already_processed = list(pd.read_csv(saving_path)["subject"])
not_processed = [sub for sub in all_subjects if sub not in already_processed]

## 10 Split Cross Validation

In [9]:
for x in range(len(already_processed), 10):
    subjects_test = all_subjects[x*13: (x+1)*13]
    subjects_train = [sub for sub in all_subjects if sub not in subjects_test]
    
    #set hyperparameters
    trainer_hyperparameter = {
        "path_train": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
        "subjects_train": subjects_train,
        "path_test": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
        "subjects_test": subjects_test,
        "acc_tester_metric": "AMI",
        "wandb": False,
        "acc_in_loop": False,
        "learning_rate": 0.001,
        "batch_size": 256,
        "margin": 0.09,
        "lr_steps": 4,
        "filter": [-4,-5,-6]
    }
    
    model_hyperparameter = {
        "sigmoid": False,
        "dropout": 0.35,
        "layers": [207, 512, 64]
    }


    filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
    #filter = None
    
    #model if needed
    if use_model:
        distance  = distances.LpDistance(p=1)
        #distance  = distances.LpDistance(p=2)
        #distance = distances.SNRDistance()
        #distance = distances.DotProductSimilarity()
        #distance = distances.CosineSimilarity()
        model = EmbeddingsModel(model_hyperparameter)
        trainer = EmbeddingTrainer(trainer_hyperparameter, model, filter=filter, distance=distance, device="cuda:0")
        trainer.trainloop(30)


    

    if use_model:
        acc = trainer.test_accuracy(max_depth=16)
    else:
        #acc testet
        acc_tester_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test
        }
        acc_tester = AccuracyTester(acc_tester_hyperparameter, filter=filter, device="cuda:0")

        acc = acc_tester.test_model(max_depth=16)

    with open(saving_path, mode='a') as result_file:
        results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        results_writer.writerow(["split_"+str(x), acc*100])

## LOSO Crossvalidation 

In [26]:
for subj in tqdm(not_processed):

    subjects_test = [subj]
    subjects_train = [sub for sub in all_subjects_intense if sub not in subjects_test]


    trainer_hyperparameter = {
        "path_train": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE\\normalized_subjects_no_outliner.pkl",
        "subjects_train": subjects_train,
        "path_test": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE\\normalized_subjects_no_outliner.pkl",
        "subjects_test": subjects_test,
        "acc_tester_metric": "AMI",
        "wandb": False,
        "acc_in_loop": False,
        "learning_rate": 0.001,
        "batch_size": 256,
        "margin": 0.09,
        "lr_steps": 20,
        "filter": [-6,-5,-4,-3,-2,-1,1,2,3,4,5,6]
    }

    model_hyperparameter = {
        "sigmoid": False,
        "dropout": 0.35,
        "layers": [207, 512, 64]
    }


    filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
    filter = None
    #model if needed
    if use_model:
        distance  = distances.LpDistance(p=1)
        #distance  = distances.LpDistance(p=2)
        #distance = distances.SNRDistance()
        #distance = distances.DotProductSimilarity()
        #distance = distances.CosineSimilarity()
        model = EmbeddingsModel(model_hyperparameter)
        trainer = EmbeddingTrainer(trainer_hyperparameter, model, filter=filter, distance=distance, device="cuda:0")
        trainer.trainloop(100)


    

    if use_model:
        acc = trainer.test_accuracy(max_depth=16)
    else:
        #acc testet
        acc_tester_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE\\normalized_subjects_no_outliner.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test
        }
        acc_tester = AccuracyTester(acc_tester_hyperparameter, filter=filter, device="cuda:0")

        acc = acc_tester.test_model(max_depth=16)

    with open(saving_path, mode='a') as result_file:
        results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        results_writer.writerow([subj, acc*100])